# Sandbox Jupyter Notebook to test main.py script
---

## Modules needed that can be load with Jupyter Lab

- xarray
- cartopy
- openpyxl

## Modules needed that can't be load with Jupyter Lab

- mapclassify 2.4.3
   - Provide in geopandas/0.12.2-foss-2022a
         - Works if we load geopandas from Jupyter Lab

Change path to current directory path

In [1]:
import os

notebook_path = os.getcwd()  # Current path
print("Directory of the notebook:", notebook_path)
notebook_path = notebook_path

os.chdir('/data/brussel/vo/000/bvo00012/vsc11137/source2suffering/') 

Directory of the notebook: /vscmnt/brussel_pixiu_data/_data_brussel/111/vsc11137


Here I copy part of luke's main.py script to test what are the needed python packages and if I can load them easily with Jupyter lab 

In [2]:
#%%  ----------------------------------------------------------------
# import and path
# ----------------------------------------------------------------

import xarray as xr
import pickle as pk
import time
# from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib as mpl
import mapclassify as mc
from copy import deepcopy as cp
import os
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import cartopy.crs as ccrs
import cartopy as cr
import geopandas as gpd
# import seaborn as sns # must comment this out for things to work on the server
scriptsdir = os.getcwd()

In [3]:
#%%------------------------------------------------------------------------------------
# flags
#--------------------------------------------------------------------------------------

# extreme event
global flags

flags = {}

flags['extr'] = 'heatwavedarea'                 # 0: all
                                                # 1: burntarea
                                                # 2: cropfailedarea
                                                # 3: driedarea
                                                # 4: floodedarea
                                                # 5: heatwavedarea
                                                # 6: tropicalcyclonedarea
                                                # 7: waterscarcity

flags['gmt'] = 'ar6_new'                        # original: use Wim's stylized trajectory approach with max trajectory a linear increase to 3.5 deg                               
                                                # ar6: substitute the linear max wth the highest IASA c7 scenario (increasing to ~4.0), new lower bound, and new 1.5, 2.0, NDC (2.8), 3.0
                                                # ar6_new: works off ar6, but ensures only 1.5-3.5 with perfect intervals of 0.1 degrees (less proc time and data volume)

flags['rm'] = 'rm'                              # no_rm: no smoothing of RCP GMTs before mapping
                                                # rm: 21-year rolling mean on RCP GMTs
 
flags['version'] = 'pickles_v3'                 # pickles: original version, submitted to Nature
                                                    # inconsistent GMT steps (not perfect 0.1 degree intervals)
                                                    # GMT steps ranging 1-4 (although study only shows ~1.5-3.5, so runs are inefficient)
                                                    # only 99.99% percentile for PIC threshold
                                                # pickles_v2: version generated after submission to Nature in preparation for criticism/review
                                                    # steps fixed in load_manip to be only 1.5-3.5, with clean 0.1 degree intervals
                                                    # 5 percentiles for PIC threshold and emergence for each
                                                # pickles_v3: version generated after the 2021 toolchains were taken away from hydra. could not longer use old pickles effectively

flags['run'] = 0                                # 0: do not process ISIMIP runs (i.e. load runs pickle)
                                                # 1: process ISIMIP runs (i.e. produce and save runs as pickle)

flags['mask'] = 0                               # 0: do not process country data (i.e. load masks pickle)
                                                # 1: process country data (i.e. produce and save masks as pickle)

flags['lifetime_exposure_cohort'] = 0           # 0: do not process ISIMIP runs to compute exposure across cohorts (i.e. load exposure pickle)
                                                # 1: process ISIMIP runs to compute exposure across cohorts (i.e. produce and save exposure as pickle)   
                                                                     
flags['lifetime_exposure_pic'] = 0              # 0: do not process ISIMIP runs to compute picontrol exposure (i.e. load exposure pickle)
                                                # 1: process ISIMIP runs to compute picontrol exposure (i.e. produce and save exposure as pickle)

flags['emergence'] = 0                          # 0: do not process ISIMIP runs to compute cohort emergence (i.e. load cohort exposure pickle)
                                                # 1: process ISIMIP runs to compute cohort emergence (i.e. produce and save exposure as pickle)

flags['birthyear_emergence'] = 0                # 0: only run calc_birthyear_align with birth years from 1960-2020
                                                # 1: run calc_birthyear_align with birth years from 1960-2100     
                        
flags['gridscale'] = 0                          # 0: do not process grid scale analysis, load pickles
                                                # 1: process grid scale analysis

flags['gridscale_le_test'] = 0                  # 0: do not process the grid scale analysis testing diff versions of constant life expectancy
                                                # 1: process grid scale analysis testing diff versions of constant life expectancy    
                                                             
flags['gridscale_country_subset'] = 0           # 0: run gridscale analysis on all countries
                                                # 1: run gridscale analysis on subset of countries determined in "get_gridscale_regions" 

flags['global_emergence_recollect'] = 1         # 0: do not load pickles of global emergence masks
                                                # 1: load pickles                  

flags['global_avg_emergence'] = 0                                                                                                
flags['gdp_deprivation'] = 0                    # 0: do not process/load lifetime GDP/GRDI average
                                                # 1: load lifetime GDP average analysis     
                                       
flags['vulnerability'] = 0                      # 0: do not process subsets of d_collect_emergence vs gdp & deprivation quantiles
                                                # 1: process/load d_collect_emergence vs gdp & deprivation quantiles for vulnerability analysis

flags['plot_ms'] = 0 # 1 yes plot, 0 no plot

flags['plot_si'] = 0

flags['reporting'] = 0  

flags['testing'] = 0   

In [4]:
#%% ----------------------------------------------------------------
# settings
# ------------------------------------------------------------------

from settings import *
ages, age_young, age_ref, age_range, year_ref, year_start, birth_years, year_end, year_range, GMT_max, GMT_min, GMT_inc, RCP2GMT_maxdiff_threshold, year_start_GMT_ref, year_end_GMT_ref, scen_thresholds, GMT_labels, GMT_window, GMT_current_policies, pic_life_extent, nboots, resample_dim, pic_by, pic_qntl, pic_qntl_list, pic_qntl_labels, sample_birth_years, sample_countries, GMT_indices_plot, birth_years_plot, letters, basins = init()

# set extremes based on flag (this needs to happen here as it uses the flags dict defined above)
set_extremes(flags)

In [5]:
#%% ----------------------------------------------------------------
# load and manipulate demographic, GMT and ISIMIP data
# ------------------------------------------------------------------

from load_manip import *

Run settings.py to define the global variables and parameters

In [6]:
run settings.py

To run the code under we need to load the openpyxl module

In [7]:
# ------------------------------------------------------------------
# Load global mean temperature projections
global df_GMT_15, df_GMT_20, df_GMT_NDC, df_GMT_strj

df_GMT_15, df_GMT_20, df_GMT_NDC, df_GMT_strj = load_GMT(
    year_start,
    year_end,
    year_range,
    flags,
)

Need to ml load geopandas/0.14.2-foss-2023a 

In [8]:
# --------------------------------------------------------------------
# Load and manipulate life expectancy, cohort and mortality data

if flags['mask']: # load data and do calculations

    print('Processing country info')

    d_countries = all_country_data(flags)

else: # load processed country data

    print('Loading processed country and region data')

    # load country pickle
    d_countries = pk.load(open('./data/{}/country_info.pkl'.format(flags['version']), 'rb'))
    
# unpack country information
df_countries = d_countries['info_pop']
gdf_country_borders = d_countries['borders']
da_population = d_countries['population_map']
df_birthyears = d_countries['birth_years']
df_life_expectancy_5 = d_countries['life_expectancy_5']
da_cohort_size = d_countries['cohort_size']
countries_regions, countries_mask = d_countries['mask']  

Loading processed country and region data


/tmp/ipykernel_103228/1641268975.py:15: UserWarning: Unpickling a shapely <2.0 geometry object. Please save the pickle again; shapely 2.1 will not have this compatibility.
  d_countries = pk.load(open('./data/{}/country_info.pkl'.format(flags['version']), 'rb'))


ModuleNotFoundError: No module named 'pandas.core.indexes.numeric'

In [ ]:
# ------------------------------------------------------------------
# load ISIMIP model data
global grid_area
grid_area = xr.open_dataarray('./data/isimip/clm45_area.nc4')

d_isimip_meta,d_pic_meta = load_isimip(
    extremes,
    model_names,
    df_GMT_15,
    df_GMT_20,
    df_GMT_NDC,
    df_GMT_strj,
    flags,
)

sims_per_step = {}
for step in GMT_labels:
    sims_per_step[step] = []
    for i in list(d_isimip_meta.keys()):
        if d_isimip_meta[i]['GMT_strj_valid'][step]:
            sims_per_step[step].append(i)